In [2]:
import os, re, time, requests, pandas as pd

In [3]:
from dotenv import load_dotenv
load_dotenv()
FRED_API_KEY = os.getenv("FRED_API_KEY")

In [4]:
FRED_BASE = "https://api.stlouisfed.org/fred/series/observations"
WB_BASE = "https://api.worldbank.org/v2/country/{iso3}/indicator/{ind}?format=json&per_page=20000"


In [ ]:
# Canonical variable names
ALIASES = {
    "Total Population": "total_population",
    "Fertility Rate": "fertility_rate",
    "Constant GDP Per Capita": "gdp_capita_const",
    "Employment-to-Population Ratio": "emp_to_pop",
    "Employment to Population Ratio": "emp_to_pop",
    "Female Labor Force Participation Rate": "flfpr",
    "Gini Index": "gini_index",
    "Net Migration": "net_migration",
}

# Allow ISO2 or ISO3 in WB URLs
ISO2_TO_ISO3 = {
    "US":"USA","CA":"CAN","DE":"DEU","SE":"SWE","PL":"POL","IN":"IND","CN":"CHN","JP":"JPN",
    "KR":"KOR","NG":"NGA","ZA":"ZAF","BR":"BRA","AU":"AUS"
}
# Also accept full country names
NAME_TO_ISO3 = {
    "United States":"USA","Canada":"CAN","Germany":"DEU","Sweden":"SWE","Poland":"POL",
    "India":"IND","China":"CHN","Japan":"JPN","South Korea":"KOR","Nigeria":"NGA",
    "South Africa":"ZAF","Brazil":"BRA","Australia":"AUS"
}

In [6]:

def normalize_iso3(s):
    s = (s or "").strip()
    if s.upper() in ISO2_TO_ISO3: return ISO2_TO_ISO3[s.upper()]
    return NAME_TO_ISO3.get(s, s.upper())

def parse_fred_series_id(url: str) -> str:
    m = re.search(r"/series/([^/?#]+)", str(url))
    if not m:
        raise ValueError(f"Cannot parse FRED series_id from: {url}")
    return m.group(1)

def fetch_fred(series_id: str) -> pd.DataFrame:
    r = requests.get(FRED_BASE, params={
        "series_id": series_id,
        "api_key": FRED_API_KEY,
        "file_type":"json"
    }, timeout=30)
    r.raise_for_status()
    obs = r.json().get("observations", [])
    df = pd.DataFrame(obs)
    if df.empty:
        return pd.DataFrame(columns=["year","value"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df["year"] = pd.to_datetime(df["date"]).dt.year
    df = df.groupby("year", as_index=False)["value"].mean()
    return df[["year","value"]]

def fetch_wb(iso3: str, indicator: str) -> pd.DataFrame:
    url = WB_BASE.format(iso3=iso3, ind=indicator)
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    js = r.json()
    if len(js) < 2 or not js[1]:
        return pd.DataFrame(columns=["year","value"])
    rows = []
    for d in js[1]:
        try:
            yr = int(d.get("date"))
        except:
            continue
        val = d.get("value")
        if val is None:
            continue
        rows.append({"year": yr, "value": float(val)})
    return pd.DataFrame(rows).sort_values("year")

def from_row(country, variable, url):
    var = ALIASES.get(variable, variable)
    # World Bank URL pattern
    if "data.worldbank.org/indicator/" in url:
        m = re.search(r"/indicator/([^?]+)\?locations=([A-Za-z]{2,3})", url)
        if not m:
            raise ValueError(f"Cannot parse WB URL: {url}")
        indicator, loc = m.group(1), m.group(2)
        iso3 = normalize_iso3(loc if len(loc)<=3 else country)
        df = fetch_wb(iso3, indicator)
        df["country"] = normalize_iso3(country)
        df["variable"] = var
        return df[["country","year","variable","value"]]
    # Otherwise FRED
    sid = parse_fred_series_id(url)
    df = fetch_fred(sid)
    df["country"] = normalize_iso3(country)
    df["variable"] = var
    return df[["country","year","variable","value"]]


In [8]:
series = pd.read_csv("data/series_list.csv")
frames = []
bad = []
for i, r in series.iterrows():
    try:
        df = from_row(r["country"], r["variable"], r["url"])
        frames.append(df)
    except Exception as e:
        bad.append({"row": i, "country": r["country"], "variable": r["variable"], "url": r["url"], "error": str(e)})

master_long = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["country","year","variable","value"])
master_long = master_long[(master_long["year"]>=1960) & (master_long["year"]<=2025)]
master_long = master_long.dropna(subset=["value"])

# Wide table
master_wide = master_long.pivot_table(index=["country","year"], columns="variable", values="value").reset_index()

# Coverage (share non-missing)
coverage = (
    master_long
    .groupby(["country","variable"])["value"]
    .apply(lambda x: x.notna().mean())
    .reset_index(name="non_missing_share")
    .sort_values(["variable","non_missing_share"], ascending=[True, False])
)

# Save
os.makedirs("data/clean", exist_ok=True)
master_long.to_csv("data/clean/master_long.csv", index=False)
master_wide.to_csv("data/clean/master_wide.csv", index=False)
coverage.to_csv("data/clean/coverage.csv", index=False)

# Log any problem rows
pd.DataFrame(bad).to_csv("data/clean/_fetch_errors.csv", index=False)
print("Done. Files in data/clean/. Problem rows logged to _fetch_errors.csv")


Done. Files in data/clean/. Problem rows logged to _fetch_errors.csv
